# Packages

In [ ]:
import cv2

from google.colab import files
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from IPython.display import clear_output, HTML
clear_output()

# Prep

In [ ]:
path = '/usr/local/share/jupyter/nbextensions/google.colab'
!cp -r {path}/* .
!rm -r {path}
!ln -s /content {path}
# change base tag

def change_base_url():
  eval_js("""
  var base = document.createElement('base')
  base.href = 'https://localhost:8080/nbextensions/google.colab/'
  document.head.prepend(base)
  """)
# make it run automatically in every cell
get_ipython().events.register('pre_run_cell', change_base_url)

fpath_img = 'gestures-sample2.jpeg'
!gdown 1HOlK7RzUrM28jNsA1qNQoqnRdh8RzKRd -O "$fpath_img"

clear_output()

# Gesture Estimation

In [ ]:
#@title Template Image
html_code = HTML('''
<!-- 相關依賴套件載入 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-core"></script>
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-converter"></script>

<!-- 本章節採用tfjs，因此載入TF.js backend -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-backend-webgl"></script>

<!-- 載入手勢辨識模型 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/hand-pose-detection"></script>

<!-- 用於呈現分析結果canvas -->
<canvas id='canvas1'></canvas>

<script>
// 宣告canvas 物件
const canvas1 = document.getElementById('canvas1');
// 宣告canvas 繪圖物件
var ctx = canvas1.getContext('2d')

// 先告js 影像物件
var myImage = new Image();
// 指定本地端範例影像路徑
myImage.src = "gestures-sample2.jpeg";
// 一旦成功載入影像，觸發執行loadImage函式
myImage.addEventListener("load", loadImage, false);

// 觸發函式
async function loadImage(e) {
  // 範例影像大小
  img_width = myImage.width;
  img_height = myImage.height;

  // 調整canvas大小
  canvas1.width = img_width;
  canvas1.height = img_height;

  // 手部關節點註解文字顏色
  ctx.strokeStyle = "red";
  // 手部關節點註解文字大小&字體
  ctx.font = "20px Arial";
  // canvas 繪製底圖
  ctx.drawImage(myImage, 0, 0, img_width, img_height);

  // 讀取模型
  const model = handPoseDetection.SupportedModels.MediaPipeHands;
  const detectorConfig = {
    runtime: 'tfjs', //指定tfjs runtime
  };
  detector = await handPoseDetection.createDetector(model, detectorConfig);

  // 模型推論分析
  const estimationConfig = {flipHorizontal: false}; // 關閉水平翻轉前處理
  const hands = await detector.estimateHands(canvas1, estimationConfig);
  console.log(hands)

  // 繪製模型分析結果
  for (var j = 0; j < hands.length; j++) {
      keypoints = hands[j].keypoints
      for (var i = 0; i < keypoints.length; i++) {
          console.log(keypoints[i]);

          // 關節點pixel-wise位置
          x = keypoints[i].x
          y = keypoints[i].y

          // 關節點類別
          name = keypoints[i].name

          // 圈選關節點估測位置
          ctx.beginPath();
          ctx.arc(x, y, 10, 0, 2 * Math.PI);
          ctx.stroke();

          // 繪製關節點文字註解
          ctx.fillText(name, x+30, y);
      }
  }
}
</script>
''')

html_code

In [ ]:
#@title Custom Image
uploaded = files.upload()
fname = list(uploaded.keys())[0]

html_code2 = HTML('''
<!-- 相關依賴套件載入 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-core"></script>
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-converter"></script>

<!-- 本章節採用tfjs，因此載入TF.js backend -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-backend-webgl"></script>

<!-- 載入手勢辨識模型 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/hand-pose-detection"></script>

<!-- 用於呈現分析結果canvas -->
<canvas id='canvas1'></canvas>

<script>
// 宣告canvas 物件
const canvas1 = document.getElementById('canvas1');
// 宣告canvas 繪圖物件
var ctx = canvas1.getContext('2d')

// 先告js 影像物件
var myImage = new Image();
// 指定本地端特定影像路徑，由外部對程式碼指定分配
myImage.src = "%s";
// 一旦成功載入影像，觸發執行loadImage函式
myImage.addEventListener("load", loadImage, false);

// 觸發函式
async function loadImage(e) {
  // 範例影像大小
  img_width = myImage.width;
  img_height = myImage.height;

  // 調整canvas大小
  canvas1.width = img_width;
  canvas1.height = img_height;

  // 手部關節點註解文字顏色
  ctx.strokeStyle = "red";
  // 手部關節點註解文字大小&字體
  ctx.font = "20px Arial";
  // canvas 繪製底圖
  ctx.drawImage(myImage, 0, 0, img_width, img_height);

  // 讀取模型
  const model = handPoseDetection.SupportedModels.MediaPipeHands;
  const detectorConfig = {
    runtime: 'tfjs', //指定tfjs runtime
  };
  detector = await handPoseDetection.createDetector(model, detectorConfig);

  // 模型推論分析
  const estimationConfig = {flipHorizontal: false}; //關閉水平翻轉前處理
  const hands = await detector.estimateHands(canvas1, estimationConfig);
  console.log(hands)

  // 繪製模型分析結果
  for (var j = 0; j < hands.length; j++) {
      keypoints = hands[j].keypoints
      for (var i = 0; i < keypoints.length; i++) {
          console.log(keypoints[i]);

          // 關節點pixel-wise位置
          x = keypoints[i].x
          y = keypoints[i].y

          // 關節點類別
          name = keypoints[i].name

          // 圈選關節點估測位置
          ctx.beginPath();
          ctx.arc(x, y, 10, 0, 2 * Math.PI);
          ctx.stroke();

          // 繪製關節點文字註解
          ctx.fillText(name, x+30, y);
      }
  }
}
</script>
''' % (fname))

html_code2

In [ ]:
#@title WebCam Image
html_code3 = HTML('''
<!-- 相關依賴套件載入 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-core"></script>
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-converter"></script>

<!-- 本章節採用tfjs，因此載入TF.js backend -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-backend-webgl"></script>

<!-- 載入手勢辨識模型 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/hand-pose-detection"></script>

<!-- 用於呈現分析結果canvas -->
<canvas id='canvas1' height="0" width="0"></canvas>

<script>
// 宣告canvas 物件
const canvas = document.getElementById('canvas1');
// 宣告canvas 繪圖物件
var ctx = canvas.getContext('2d')

// webcam 攝影
async function takePhoto() {
  // 讀取模型
  const model = handPoseDetection.SupportedModels.MediaPipeHands;
  const detectorConfig = {
    runtime: 'tfjs', //指定tfjs runtime
  };
  detector = await handPoseDetection.createDetector(model, detectorConfig);

  const div = document.createElement('div'); // 空行
  const capture = document.createElement('button'); // 攝影按鈕
  capture.textContent = 'Capture'; // 攝影按鈕文字
  div.appendChild(capture);

  const video = document.createElement('video'); // 宣告影片物件
  video.style.display = 'block';
  const stream = await navigator.mediaDevices.getUserMedia({video: true}); // 宣告串流物件

  document.body.appendChild(div);
  div.appendChild(video);
  video.srcObject = stream; // 使影片物件顯示串流內容
  await video.play();

  // 調整Video Element 大小
  google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

  // 等待點擊攝影按鈕
  await new Promise((resolve) => capture.onclick = resolve);

  // canvas 繪製底圖
  canvas.width = video.videoWidth;
  canvas.height = video.videoHeight;
  ctx.drawImage(video, 0, 0);

  // 停止 webcamera
  stream.getVideoTracks()[0].stop();
  div.remove();

  // 模型推論分析
  const estimationConfig = {flipHorizontal: false}; //關閉水平翻轉前處理
  const hands = await detector.estimateHands(canvas, estimationConfig);
  console.log(hands)

  // 手部關節點註解文字顏色
  ctx.strokeStyle = "red";
  // 手部關節點註解文字大小&字體
  ctx.font = "20px Arial";
  // 繪製模型分析結果
  for (var j = 0; j < hands.length; j++) {
      keypoints = hands[j].keypoints
      for (var i = 0; i < keypoints.length; i++) {
          console.log(keypoints[i]);

          // 關節點pixel-wise位置
          x = keypoints[i].x
          y = keypoints[i].y

          // 關節點類別
          name = keypoints[i].name

          // 圈選關節點估測位置
          ctx.beginPath();
          ctx.arc(x, y, 10, 0, 2 * Math.PI);
          ctx.stroke();

          // 繪製關節點文字註解
          ctx.fillText(name, x+30, y);
      }
  }
}

takePhoto()
</script>
''')

html_code3

In [ ]:
#@title WebCam Video
html_code4 = HTML('''
<head>
  <!-- 相關依賴套件載入 -->
  <script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-core"></script>
  <script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-converter"></script>

  <!-- 本章節採用tfjs，因此載入TF.js backend -->
  <script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-backend-webgl"></script>

  <!-- 載入手勢辨識模型 -->
  <script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/hand-pose-detection"></script>

  <!-- 隱藏html video -->
  <style>
      video{
          visibility: hidden;
      }
  </style>
</head>

<!-- 呈現結果的canvas -->
<canvas id="detect_result"></canvas>
<!-- 串流video -->
<video autoplay playsinline muted id="webcam" ></video>

<script>
async function app() {
  // 讀取模型
  const model = handPoseDetection.SupportedModels.MediaPipeHands;
  const detectorConfig = {
    runtime: 'tfjs', //指定tfjs runtime
  };
  detector = await handPoseDetection.createDetector(model, detectorConfig);

  const webcamElement = document.getElementById('webcam'); // 宣告影片物件

  // 宣告canvas物件
  const canvas = document.getElementById('detect_result');
  const ctx = canvas.getContext('2d');

  let showResult = async function () {
    // 模型推論分析
    const estimationConfig = {flipHorizontal: false}; //關閉水平翻轉前處理
    const hands = await detector.estimateHands(canvas, estimationConfig);

    // canvas 繪製底圖
    canvas.width = webcamElement.videoWidth;
    canvas.height = webcamElement.videoHeight;
    ctx.drawImage(webcamElement, 0, 0);

    // 手部關節點註解文字顏色
    ctx.strokeStyle = "red";
    // 手部關節點註解文字大小&字體
    ctx.font = "20px Arial";
    // 繪製模型分析結果
    for (var j = 0; j < hands.length; j++) {
        keypoints = hands[j].keypoints
        for (var i = 0; i < keypoints.length; i++) {
            console.log(keypoints[i]);

            // 關節點pixel-wise位置
            x = keypoints[i].x
            y = keypoints[i].y

            // 關節點類別
            name = keypoints[i].name

            // 圈選關節點估測位置
            ctx.beginPath();
            ctx.arc(x, y, 10, 0, 2 * Math.PI);
            ctx.stroke();

            // 繪製關節點文字註解
            ctx.fillText(name, x+30, y);
        }
    }

    // 設定每偵顯示秒數
    setTimeout(function () {
        showResult();
    }, 300);
  }
  let setupWebcam = function () {
      return new Promise((resolve, reject) => {
          const navigatorAny = navigator;
          navigator.getUserMedia = navigator.getUserMedia ||
              navigatorAny.webkitGetUserMedia || navigatorAny.mozGetUserMedia ||
              navigatorAny.msGetUserMedia;
          if (navigator.getUserMedia) {
              navigator.getUserMedia({
                      video: true
                  },
                  (stream) => {
                      webcamElement.srcObject = stream;
                      webcamElement.addEventListener('loadeddata', () => resolve(),
                          false);
                  },
                  (err) => reject(err));
          } else {
              reject("getUserMedia failed");
          }
      });
  }
  setupWebcam().then(
      () => {
          showResult();
      },
      (err) => {
          console.log(err);
      }
  )
}
app();
</script>
''')

html_code4